<table align="left" style="border-style: hidden" class="table"> <tr><td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Fall 2018</h4><p>Ani Adhikari and Jim Pitman</p>CC BY-NC 4.0</div></td></tr></table><!-- not in pdf -->

In [ ]:
# SETUP

import itertools
import numpy as np
from datascience import *
from prob140 import *

# These lines do some fancy plotting magic
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Useful for probability calculations
from scipy import stats
from scipy import misc

In [ ]:
def _remove_gaps(domain, prob):
    """
    domain must be sorted
    """
    new_probs = []
    index = 0
    max_value = max(domain)
    for i in range(max_value + 1):
        if i < domain[index]:
            new_probs.append(0)
        else:
            new_probs.append(prob[index])
            index += 1
    return np.array(new_probs)

In [ ]:
def plot_sample(p_array, s, m):
    """
    Plots a dot plot sampled s times from p_array.
    
    Parameters
    ----------
    p_array : array
        An array of probabilities corresponding to p_0, ..., p_n.
    s : int
        Sample size.
    m : int
        Highlighted value.
    """
    assert s % 2 == 1, 's must be odd.'
    n = len(p_array)
    samples = sorted(np.random.choice(np.array(n), size=s, p=p_array))
    counts = _remove_gaps(*np.unique(samples, return_counts=True))
    for i in range(n):
        if i < len(counts):
            x = [i] * counts[i]
            y = np.arange(counts[i])
            if i == m:
                color = 'k'
                plt.vlines(i, -1, max(10, max(counts)), linewidth=1,
                           color='blue',lw=2, label='m')
            else:
                color = 'k'
            plt.scatter(x, y, color=color, s=40)
    median_value = samples[s // 2]
    # If there multiple instance of the median value, want to know which
    # one is the real median.
    median_index = s // 2 - samples.index(median_value)
    plt.scatter(median_value, median_index, color='r', s=40,
                label='median')
    plt.axhline(-1, color='k')
    plt.xlim(-0.5, n - 0.5)
    ax = plt.gca()
    ax.set_xticks(np.arange(n))
    ax.set_yticks([])
    ax.grid(False)
    plt.ylim(-1, max(10, max(counts)))
    plt.legend()
    if median_value > m:
        print('Sample median is greater than m.')
    else:
        print('Sample median is not greater than m.')
    plt.title('Sample Median')


## Lab Resources

* [`prob 140` Library Documentation](http://prob140.org/prob140/)
* [Data 8 Python Reference](http://data8.org/fa18/python-reference.html)
* [Prob 140 Code Reference Sheet](http://prob140.org/assets/prob140_code_reference.pdf)
* [`scipy.stats` Documentation](https://docs.scipy.org/doc/scipy/reference/stats.html)

# Lab 4: Expectations of Discrete Order Statistics #
Additivity is a powerful property of expectation: you have seen several examples in which writing a random variable as a sum of simpler variables has led to a simple calculation of its expected value. But other methods are needed for finding the expectations of random variables that can't easily be written as sums. 

Using the basic definition of expectation is one such method, and it works well when you can easily write down the distribution of the random variable and have the computational tools to work out the resulting sum. However, distributions aren't always straightforward to write down.

This lab is about expectations of a class of random variables that don't have natural representations as sums of simpler variables. These variables, called *order statistics*, can be thought of as the elements of a random sample arranged in increasing order. We will work with just two of them:
- the minimum
- the median

You have used the sample median as a statistic in Data 8. How big is a random sample median expected to be? By the end of this lab you will have an answer to this question. 

The corresponding question for the sample mean has an easy answer by additivity: the expectation of a random sample mean is the population mean. But additivity isn't very helpful for medians as medians aren't easy to think of as sums. A different approach is needed. 

What you will learn in this lab:
- How to compute expectations using the tail sum formula
- How to find the expected minimum value in a random sample
- How to calculate tail probabilities for a random sample median
- How to find the expectation of a random sample median
- How random sample medians behave as the sample size gets large

Let's start by reviewing a clever way of finding the expectation of a non-negative integer valued random variable.

## Part 1: Tail Sum Formula for Expectation ##
Let $X$ be a random variable with values $0, 1, 2, 3, \ldots, N$ for some fixed integer $N$. As you know, the cumulative distribution function (cdf) of $X$ is the function $F$ defined by
$$
F(x) ~ = ~ P(X \le x) 
$$
for all $x$.

The *tail probability function* $T$ is defined by

$$
T(x) ~ = ~ P(X > x) ~ = ~ 1 - F(x)
$$
for all $x$.

Here is the probability histogram of a random variable with distribution `example_dist`. The gold area is equal to $F(4)$ and the blue area is $T(4)$.

In [ ]:
example_vals = np.arange(8)
example_probs = make_array(0.05, 0.1, 0.2, 0.25, 0.15, 0.12, 0.08, 0.05)

example_dist = Table().values(example_vals).probability(example_probs)

Plot(example_dist, event = np.arange(5))

plt.title('Gold Area = $F(4)$, Blue Area = $T(4)$');

### Tail Sum Formula ###

For simplicty of notation, define $p_i = P(X = i)$ for $0 \le i \le N$. Then $p_i \ge 0$ for all $i$ and $\sum_{i=0}^N p_i = 1$.

The definition of the expectation of $X$ is

$$
E(X) ~ = ~ p_1 + 2p_2 + 3p_3 + \cdots + Np_N
$$

As you have seen, the *tail sum formula* allows you to calculate the expectation of a non-negative integer valued random variable based on the tail probabilities alone:

$$
E(X) = T(0) + T(1) + \cdots + T(N-1) + T(N)
$$

In the sum above, the last term $T(N)$ is 0. So you can stop the sum at $T(N-1)$, but it doesn't hurt to include $T(N)$.

### 1a) Computing Tail Probabilities ###
For a random variable $X$ with possible values $0, 1, 2, \ldots, N$ as above, we will use the term "probability array" for an array consisting of the sequence $p_0, p_1, \ldots, p_N$. 

Define a function `tails` that takes the probability array as its argument and returns an array of all the tail probabilities. Remember the word "cumulative" in cdf, and use `np.cumsum` appropriately.

In [ ]:

def tails(prob_array):
    return ...

Suppose $p_0 = 0.3$, $p_1 = 0.5$, and $p_2 = 0.2$, and let `probs` be the array [0.3, 0.5, 0.2].

What should `tails(probs).item(0)` and `tails(probs).item(2)` evaluate to? Check that they work out as they should.

In [ ]:
probs = make_array(0.3, 0.5, 0.2)

tails(probs).item(0), tails(probs).item(2)

### 1b) Expectation by the Definition ###
Here is the distribution of a random variable $X$.

In [ ]:
values_X = np.arange(5)
probs_X = make_array(0.3, 0.35, 0.2, 0.1, 0.05)
dist_X = Table().values(values_X).probability(probs_X)
dist_X

Use array operations and the arrays defined above to find $E(X)$. The last line of code should evaluate to $E(X)$.

In [ ]:

...

The `prob140` method `ev` operates on a Distribution object and returns the expectation. Run the cell to check if your calculation above is correct.

In [ ]:
dist_X.ev()

### 1c) Expectation by the Tail Sum Formula ###
The values of $X$ are non-negative integers, so the tail sum formula can be used to find $E(X)$. Write one line of code that uses the function `tails` from part **a** and evaluates to $E(X)$ by applying the tail sum formula.

In [ ]:

...

#newpage

## Part 2: Sample Minimum ##
Let $X_1, X_2, \ldots, X_n$ be i.i.d., each with possible values $0, 1, 2 \ldots N$. Let $Y$ be the smallest of the $n$ sampled values, that is,

$$
Y ~ = ~ \min\{X_1, X_2, \ldots , X_n\}
$$

### 2a) [On Paper] Expectation of the Sample Minimum ###

Let $T_Y$ be the tail probability function of $Y$, that is, for all $k$ let $T_Y(k) = P(Y > k)$. Let $T_X$ be the tail probability function of $X_1$. For each $k$, find a formula for $T_Y(k)$ in terms of $T_X(k)$, and hence find a formula for $E(Y)$ in terms of $T_X$. Your formulas can involve $N$ and $n$.

### 2b) Computing the Expected Sample Minimum ###
Now implement the result you found in part **a**. Define a function `expected_sample_min` that takes the probability array of $X_1$ as the first argument and $n$ as the second, and returns $E(Y)$. Use the function `tails` that you defined in Part 1.

In [ ]:

def expected_sample_min(prob_array, n):
    ...

### 2c) Checking that Your Function Works ###

Let $X_1$ and $X_2$ be i.i.d. uniform on 0, 1, and 2, and let $Y = \min(X_1, X_2)$. Use the lines of the cell below to compute the distribution of $Y$ (without using tails; just enumerate the outcomes) and hence $E(Y)$ (by applying the definition). The last line of code should evaluate to $E(Y)$.

In [ ]:

# P(Y = 0)
p_y_0 = ...

# P(Y = 1)
p_y_1 = ...

# P(Y = 2)
p_y_2 = ...

# E(Y)
...

Now check that your function works. Start by defining the probability array you need as the first argument. The last line of code should evaluate to the answer you got above.

In [ ]:

probs = ...
expected_sample_min(probs, 2)

### 2d) Expected Minimum of Rolls of a Die ###
The expectation of the minimum of two rolls of a die is about 2.528, as you should check by hand when you get a chance. To check this by using your function `expected_sample_min`, what probability array should you use? Be careful ...

Fill in the comment with your reasoning. Assign `fair_die_prob_array` to the probability array, and run the cell. The last line should evaluate to approximately 2.528. 

In [ ]:
"""
The array needs to include p_0 which is the chance of ...
p_0 = ...
"""

fair_die_prob_array = ...
expected_sample_min(fair_die_prob_array, 2)

Write one line of code that evaluates to the expected minimum of 10 rolls of a die.

In [ ]:

...

Explain why this answer is less than the expected minimum of two rolls of a die.


**Your answer here.**

### 2e) Sample Minimum Household Size ###
The website Statista provides the [distributions of household sizes](https://www.statista.com/statistics/242189/disitribution-of-households-in-the-us-by-household-size/) in the United States, for various years. Go to the website and notice their choice of stacked bars representing the proportions of households of different sizes. If you hover your cursor over one of the stacks, you can see the percents in that stack. Notice also:
- No household has 0 members, so the bottom bar (blue) represents households with one member.
- The distributions have been *truncated* at 7. That is, the last category of sizes is "7 or more". This truncation could markedly affect the calculation of average household size, so we won't do that. However, it will have little effect on the minimum household size in the sample and on the sample median. 

Let's start with the 2017 distribution. We'll call the final category "7 persons", not "7 or more". So everything we do using these proportions will be an approximation, not an exact value.

First, run the cell below for some cleanup to compensate for the fact that the proportions provided don't quite add up to 1. The array `hh_size_2017` is the household size probability array for 2017: there are no households with 0 members, roughly 28% with 1 member, and so on. We have just normalized Statista's proportions by the total and rounded the results.

In [ ]:
statista = make_array(0.2792, 0.3447, 0.1546, 0.1284, 0.058, 0.0221, 0.0128)
hh_size_2017 = np.append(0, statista)
hh_size_2017 = np.round(hh_size_2017/sum(hh_size_2017), 4)
hh_size_2017

Use your function `expected_sample_min` to write one line of code that evaluates to the expected minimum household size in a random sample of 100 households from 2017.

In [ ]:

...

Does the answer agree with your intuition? Explain


**Your answer here.**

Now do the same using the 1970 distribution. We have set up the 1970 household size probability array.

In [ ]:
hh_size_1970 = np.append(0, make_array(.17, .29, .17, .16, .1, .06, .05))

In [ ]:

...

#newpage

## Part 3: Tail Probabilities of the Sample Median ##
We will analyze the sample median in the straightforward case where the sample size $s = 2n+1$ is odd and "the median" is defined as the $(n+1)$st value when the sample is sorted in ascending order. The $(n+1)$st is the right item to pick out, because $2n+1 = n + 1 + n$.

This sample median is formally known as the *$(n+1)$st order statistic* of the sample.

The example below will help clarify the definition.

In [ ]:
# 7 = 2*3 + 1

sample_7 = make_array(23, 81, 34, 13, 56, 27, 26)
sample_7_median = np.sort(sample_7).item(3)       # item(3) is the 4th item
sample_7_median

### 3a) The Event "Sample Median $ > m$" ###
Start by visualizing what has to happen for the sample median to be larger than a given value. The function `plot_sample` takes three arguments:
- a probability array of our usual form $[p_0, p_1, \ldots, p_N]$
- an odd numbered sample size $s = 2n+1$ for some non-negative integer $n$
- a value of $m$

It displays a dot plot of an i.i.d. sample of size $s$ drawn from the probability array (remember that the possible values are $0, 1, \ldots, N$). The vertical blue line is at $m$. The statement above the plot says whether or not the sample median is greater than $m$.

Run the cell below many times just as written. Each time, count how many dots are on the right of the vertical line, and note whether or not the median is bigger than $m$. Then change $m$ and $s$ and run it again a few times, but don't go too crazy with the changes.

In [ ]:
plot_sample(hh_size_1970, 7, 2)

Fill in the blank with the appropriate phrase:

The median of a sample of size $2n+1$ is bigger than $m$ if and only if $\underline{~~~~~~~~~~~~~~~~}$ of the sampled elements are bigger than $m$.


**Your answer here.**

### 3b) [On Paper] Tails of the Sample Median ###
Let $s = 2n+1$. Let $X_1, X_2, \ldots , X_s$ be i.i.d., each with possible values $0, 1, 2, \ldots, N$ and tail probability function $T_X$. 

Let $M$ be the median of $X_1, X_2, \ldots, X_s$, as defined above. 

Fill in the first blank below with the name of a famous distribution and the appropriate parameters. Fill in the second blank with an integer expressed in terms of $s$. **Explain your choices.**

For an integer $m$ in the range 0 through $N$, $P(M > m)$ is equal to the probability that a random variable with $\underline{~~~~~~~~~~~~~~~~~~~~~}$ distribution is greater than $\underline{~~~~~~~~~~~~}$.

[Hint: Think about what you counted in part **a**. Define the $X$'s as trials and figure out how to define "success".]

#newpage

## Part 4. Computing the Expected Sample Median ##

The goal of this part is to write a function that computes the expectation of a random sample median using the formula you derived in Part 3. 

For this, it will help to have two computational preliminaries.

### 4a) Odd or Even? ###
The Python operator `%` operates on two numbers `a` and `b` where `b` is non-zero, and returns the remainder after `a` is divided by `b`. The expression

`a % b`

evaluates to the remainder of `a` divided by `b`.

For example, the remainder of 14 divided by 4 is 2, because $14 = (4*3) + 2$.

In [ ]:
14 % 4

Use `%` to define a function `is_odd` that takes non-negative integer as its argument and returns `True` if the argument is odd and `False` otherwise.

In [ ]:

def is_odd(n):
    ...

### 4b) Binomial CDF and Tails ###
Let $n$ and $k$ be integers such that $0 \le k \le n$, and let $p \in (0, 1)$.

You know that `stats.binom.pmf(k, n, p)` evaluates to $\binom{n}{k} p^k (1-p)^{n-k}$, the probability mass of the binomial $(n, p)$ distribution at the value $k$.

Also, `stats.binom.cdf(k, n, p)` evaluates to the cumulative distribution function (cdf) of the binomial $(n, p)$ distribution, evaluated at $k$. That is, it evaluates to $P(W \le k)$ where $W$ has the binomial $(n, p)$ distribution.

As a numerical example, let $W$ have the binomial $(100, 0.7)$ distribution. Use `stats.binom.cdf` to write an expression that evaluates to the tail probability $P(W > 75)$.

In [ ]:

...

To get the cdf of the binomial $(n, p)$ distribution at an array `x` of possible values, use `stats.binom.cdf(x, n, p)`.

For example, here is the cdf of the binomial (100, 0.5) distribution evaluated at the points 45, 50, and 55.

In [ ]:
stats.binom.cdf(make_array(45, 50, 55), 100, 0.5)

When you use `stats.binom` methods, you can replace other arguments by arrays too. For example, suppose you have three biased coins that land heads with chance 0.2, 0.3, and 0.4, and for each of them you want the chance of getting 4 heads in 7 tosses. The expression in the cell below evaluates to an array containing the three chances.

In [ ]:
stats.binom.pmf(4, 7, make_array(0.2, 0.3, 0.4))

I have 9 coins. For each $i$ in the range 1 through 9, Coin $i$ lands heads with chance $p_i = i/10$. Write an expression that evaluates to an array consisting of the following tail probabilities:

$$
P(\text{more than 3 heads in 8 tosses of Coin } i), ~~~ i = 1, 2, 3, \ldots, 9
$$

In [ ]:

...

### 4c) Expectation of the Sample Median ###
You are now ready to compute the expectation of a random sample median.

Define a function `expected_sample_median` that takes as its arguments a probability array of the form $[p_0, p_1, \ldots, p_N]$ and a sample size, and returns:
- the string `'For this calculation, the sample size has to be an odd number.'` if the sample size is even
- the expectation of the median of an i.i.d. sample of the given size from the given array, if the sample size is odd

You will need:

- The tail sum formula
- Your answer to **3b**
- The computing preliminaries **4a** and **4b**

In [ ]:

def expected_sample_median(prob_array, s):
    ...

Run the cell below to confirm that your function is doing the right thing. It uses the probability array `probs_X` from **1b**.

In [ ]:
expected_sample_median(probs_X, 10)

If the sample size is 1, then there's just one element $X_1$ in the sample, and that's the median. So if the sample size is 1 then $M = X_1$ and hence $E(M) = E(X_1)$. 

The cell below uses the probability array `probs_X` from **1b**. Check that your function does the right thing in the cell below. The value you should get is in **1b**.

In [ ]:
expected_sample_median(probs_X, 1)

### 4d) Sample Median Household Size ###

Find the expectation of the median household size in a random sample of 25 households taken in 1970.

In [ ]:

...

Do the same for a random sample of 25 households in 2017.

In [ ]:

...

### 4e) Change Over Time ###
Look at [the graphs](https://www.statista.com/statistics/242189/disitribution-of-households-in-the-us-by-household-size/) again. The distribution of household size in the US has been changing over the decades.

Fill in the blanks with numbers, and explain your reasoning:

As the sample size increases, the expected median household size in the sample from 1970 will get closer to $\underline{~~~~~~~~~~~~~~~~~~}$ and the expected median household size in the sample from 2017 will get closer to $\underline{~~~~~~~~~~~~~~~~~~}$.


**Your answer here.**

Check that your answers above are consistent with the computed expected sample medians. In the cell below, enter two expressions. The first should evaluate to the expected median household size in a random sample of size 1001 in 1970, and the second should evaluate to the expected median household size in a random sample of size 1001 in 2017.

In [ ]:

... , ...

## Conclusion ##
What you have learned in this lab:

- The use of the tail sum formula in finding the expectations of non-negative integer valued random variables
- Some properties of *discrete order statistics*, that is, sorted values in random samples
- How to find the expectation of the minimum of an i.i.d. sample from a population of non-negative integers
- A duality between the tail of the median of an i.i.d. sample and the tail of a well known distribution
- The use of this duality and the tail sum formula in finding the expectation of the median of an i.i.d. sample

That's pretty impressive! Congratulations.

## Submission Instructions

1. **Save your notebook using File > Save and Checkpoint.**
2. Run the cell below to generate a pdf file.
3. Download the pdf file and confirm that none of your work is missing or cut off.
4. Submit the written component to **Lab04a** and the code component to **Lab04b** on Gradescope.

Written questions:
* 2a
* 3b

Code questions:
* 1a-c
* 2b-e
* 3a
* 4a-e

#### Logistics 

1. Examine the generated pdf before uploading to make sure that it contains all of your work.
2. When submitting to Gradescope, select the pages of your upload corresponding to each question. 
3. If you encounter any difficulties when submitting or exporting your assignment, please make a private Piazza post **before the deadline**. 

### **We will not grade assignments which do not have pages selected for each question or were submitted after the deadline.** 

In [ ]:
import gsExport
gsExport.generateSubmission("Lab04.ipynb")